# NEURAL NETWORKS

### 신경망의 일반적인 교육 절차

- 학습 가능한 매개변수(또는 가중치)가 있는 신경망 정의

- 입력 데이터 집합을 반복

- 네트워크를 통한 프로세스 입력

- 손실 계산

- 그레디언트를 역전파



- 일반적으로 사용하는 네트워크 가중치 업데이트 규칙 : . 가중치 = 가중치 - 학습 속도 * 그레디언트

### 신경망 정의

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F


In [2]:
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__() # nn.Module의 생성자를 호출하여 초기화

        # 1개의 입력 이미지 채널, 6개의 출력 채널, 5x5 크기의 컨볼루션 필터
        self.conv1 = nn.Conv2d(1, 6, 5) 

        # 6개의 입력 이미지 채널, 16개의 출력 채널, 5x5 크기의 컨볼루션 필터
        self.conv2 = nn.Conv2d(6, 16, 5) 

        # 선형 연산: 입력 이미지를 1차원으로 평탄화하여 16 * 5 * 5의 크기를 갖는 벡터로 변환
        # 이후 120개의 출력 뉴런으로 이루어진 완전 연결 층
        self.fc1 = nn.Linear(16 * 5 * 5, 120) 

        # 120개의 입력 뉴런, 84개의 출력 뉴런으로 이루어진 완전 연결 층
        self.fc2 = nn.Linear(120, 84) 

        # 84개의 입력 뉴런, 10개의 출력 뉴런으로 이루어진 완전 연결 층
        self.fc3 = nn.Linear(84, 10) 


    def forward(self, x):  # forward 메서드는 순전파 과정 정의
        # (2, 2) 윈도우에 대한 최대 풀링
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # 이미지 크기가 정사각형인 경우, 단일 숫자로 크기 지정 가능
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)  # 배치 차원을 제외한 모든 차원을 평탄화
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)


Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
# Net 모델 파라미터 확인
params = list(net.parameters())
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 5, 5])


In [4]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0636, -0.0127,  0.0282,  0.0645,  0.0560,  0.0924, -0.0557,  0.0626,
          0.1369, -0.0121]], grad_fn=<AddmmBackward0>)


In [5]:
net.zero_grad()
out.backward(torch.randn(1, 10))

### 손실 함수

In [6]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.9976, grad_fn=<MseLossBackward0>)


In [7]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

### 역전파

In [8]:
net.zero_grad()     # zeroes the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0016,  0.0023, -0.0093,  0.0130,  0.0007, -0.0219])


### 가중치 업데이트

In [10]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [11]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update